In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [3]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from torch.utils.data import DataLoader
from datasets import Dataset
from transformers import DataCollatorWithPadding
from torch.optim import Adam
from tqdm.auto import tqdm, trange

ModuleNotFoundError: No module named 'datasets'

In [4]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [3]:
pd.options.display.max_colwidth = 300

In [4]:
df_dev = pd.read_csv('in_domain_dev.csv').rename(columns={'acceptable':'label'})
df_train = pd.read_csv('in_domain_train.csv').rename(columns={'acceptable':'label'})

In [5]:
df_train.sample()

,id,sentence,label,error_type,detailed_source
7797,7797,Вместе с тем систематическое исследование каки...,0,Semantics,Paducheva2013


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7869 entries, 0 to 7868
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               7869 non-null   int64 
 1   sentence         7869 non-null   object
 2   label            7869 non-null   int64 
 3   error_type       7869 non-null   object
 4   detailed_source  7869 non-null   object
dtypes: int64(2), object(3)
memory usage: 307.5+ KB


In [7]:
df_train.iloc[:,2:4].value_counts()

label  error_type
1      0             5864
0      Syntax        1068
       Semantics      805
       Morphology     132
Name: count, dtype: int64

In [ ]:
base_model = 'ai-forever/ruRoberta-large'
#base_model = 'ai-forever/ruBert-base'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>'}

In [ ]:
data = Dataset.from_pandas(df_train[['sentence', 'label']]).train_test_split(test_size=0.2, seed=42)
data_tokenized = data.map(lambda x: tokenizer(x['sentence'], truncation=True, max_length=512), batched=True, remove_columns=['sentence'])

Map:   0%|          | 0/6295 [00:00<?, ? examples/s]

Map:   0%|          | 0/1574 [00:00<?, ? examples/s]

In [ ]:
data_tokenized

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 6295
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1574
    })
})

In [ ]:
collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader = DataLoader(data_tokenized['train'], shuffle=True, batch_size=32, collate_fn=collator)
val_dataloader = DataLoader(data_tokenized['test'], shuffle=False, batch_size=32, collate_fn=collator)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
if torch.cuda.is_available():
    model.cuda()

optimizer = Adam(model.parameters(), lr=1e-6)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
losses = []
for epoch in trange(10):
    pbar = tqdm(train_dataloader)
    model.train()
    for i, batch in enumerate(pbar):
        out = model(**batch.to(model.device))
        out.loss.backward()
        if i % 1 == 0:
            optimizer.step()
            optimizer.zero_grad()
        losses.append(out.loss.item())
        pbar.set_description(f'loss: {np.mean(losses[-100:]):2.2f}')
    model.eval()
    eval_losses = []
    eval_preds = []
    eval_targets = []
    for batch in tqdm(val_dataloader):
        with torch.no_grad():
                out = model(**batch.to(model.device))
        eval_losses.append(out.loss.item())
        eval_preds.extend(out.logits.argmax(1).tolist())
        eval_targets.extend(batch['labels'].tolist())
    print('recent train loss', np.mean(losses[-100:]), 'eval loss', np.mean(eval_losses), 'accuracy', np.mean(np.array(eval_targets) == eval_preds))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/197 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

recent train loss 0.5734630849957466 eval loss 0.5530033135414123 accuracy 0.7490470139771284


  0%|          | 0/197 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

recent train loss 0.29303546145558357 eval loss 0.5494240608811378 accuracy 0.8341804320203303


  0%|          | 0/197 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

recent train loss 0.2611909595876932 eval loss 0.6133131563663483 accuracy 0.8303684879288437


In [ ]:
data_dev = Dataset.from_pandas(df_dev[['sentence', 'label']])
data_tokenized_dev = data_dev.map(lambda x: tokenizer(x['sentence'], truncation=True, max_length=512), batched=True, remove_columns=['sentence'])

In [ ]:
data_val_dataloader = DataLoader(data_tokenized_dev, shuffle=False, batch_size=128, collate_fn=collator)

In [ ]:
model.eval()
eval_losses = []
eval_preds = []
eval_targets = []
for batch in tqdm(data_val_dataloader):
    with torch.no_grad():
            out = model(**batch.to(model.device))
    eval_losses.append(out.loss.item())
    eval_preds.extend(out.logits.argmax(1).tolist())
    eval_targets.extend(batch['labels'].tolist())
print('recent train loss', np.mean(losses[-100:]), 'eval loss', np.mean(eval_losses), 'accuracy', np.mean(np.array(eval_targets) == eval_preds))

In [ ]:
print(f'f1_score = {f1_score(eval_targets, eval_preds)}, prec= {precision_score(eval_targets, eval_preds)}, recall = {recall_score(eval_targets, eval_preds)}')

Найдем примеры ошибок

In [ ]:
df_result = pd.concat([df_dev['sentence'], pd.DataFrame(eval_targets, columns=['target']), pd.DataFrame(eval_preds, columns=['pred'])], axis=1)
df_result.head()

In [ ]:
df_result[df_result['target'] != df_result['pred']]


Возьмите RuGPT3 base или large и решите данное задание с помощью методов few-/zero-shot

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

import gc
gc.collect()
torch.cuda.empty_cache()

NameError: name 'torch' is not defined